In [1]:
import json
import numpy as np
import spglib as spg

In [2]:
data = {}
for hno in range(1, 531):
    sdata = spg.get_spacegroup_type(hno)
    intsym = sdata['international_short']
    data[hno] = {'spg': sdata}
    data[hno]['ops'] = [a.tolist() for a in spg.get_symmetry_from_database(hno).values()]

In [3]:
# Now establish a hierarchy
cmat = np.zeros((530, 530)).astype(int)

opsets = []
for hno, d in data.items():
    R = d['ops'][0]
    T = d['ops'][1]
    
    opsets.append(set([(tuple(np.array(r).reshape(-1)), tuple(t)) for r, t in zip(R, T)]))

for i1, s1 in enumerate(opsets):
    sl = 0
    for i2, s2 in enumerate(opsets[i1+1:]):
        if s2.issuperset(s1):
            cmat[i1, i1+i2+1] = 1
            sl += 1

sublv = np.sum(cmat, axis=1)

In [4]:
# Split them by levels
layers = []
layerno = np.zeros(530).astype(int)
for i, n in enumerate(sorted(list(set(sublv)))):
    inds = np.where(sublv == n)[0]
    layers.append(inds)
    layerno[inds] = i

In [9]:
# Find the connections
tree = [{'children': [], 'parent': None} for i in range(530)]
for i, l in enumerate(layers):
    for i1 in l:
        supersets = np.where(cmat[:,i1] > 0)[0]
        if len(supersets) > 0:
            p = supersets[np.argmin(layerno[supersets])]
            tree[i1]['parent'] = int(p)+1
            tree[p]['children'].append(int(i1)+1)

In [10]:
# Save all the data
for hno, d in data.items():
    d['children'] = tree[hno-1]['children']
    d['parent'] = tree[hno-1]['parent']

In [11]:
json.dump(data, open('../data/spgtree.json', 'w'))